### part1 : train lightgbm model

In [1]:
import os, math, joblib
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [2]:

MODELS_DIR = os.path.join("outputs", "models")
os.makedirs(MODELS_DIR, exist_ok=True)

QUANTILES = [0.1, 0.5, 0.9]
FEATURES = ["Lag_1","Lag_2","Lag_3","RollingMean_3","RollingStd_3","USD_Trend","Holiday_Ratio","Week_Index","Category"]
TARGET = "Demand"

In [3]:
df = pd.read_excel('data.xlsx')
print("Loaded rows:", len(df), "columns:", df.columns.tolist())

Loaded rows: 870 columns: ['SKU-index', 'Category', 'Week_Index', 'Demand', 'Lag_1', 'Lag_2', 'Lag_3', 'RollingMean_3', 'RollingStd_3', 'Holiday_Ratio', 'USD_Trend']


In [4]:
train_df = df[df['Week_Index'] <= 18].copy()
pred_df  = df[df['Week_Index'] == 19].copy()
print("Train rows:", len(train_df), "Predict rows:", len(pred_df))



Train rows: 726 Predict rows: 48


In [5]:
features = [c for c in FEATURES if c in train_df.columns]
print("Using features:", features)


Using features: ['Lag_1', 'Lag_2', 'Lag_3', 'RollingMean_3', 'RollingStd_3', 'USD_Trend', 'Holiday_Ratio', 'Week_Index', 'Category']


In [6]:
X = train_df[features]
y = train_df[TARGET]
X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.12, random_state=RANDOM_SEED, shuffle=True)
dtrain = lgb.Dataset(X_tr, label=y_tr)
dvalid = lgb.Dataset(X_val, label=y_val)
params_reg = {"objective":"regression","metric":"rmse","learning_rate":0.05,"num_leaves":31,"verbosity":-1,"seed":RANDOM_SEED}
bst = lgb.train(
    params_reg,
    dtrain,
    num_boost_round=1000,
    valid_sets=[dtrain, dvalid],
)

val_pred = bst.predict(X_val)
print("Quick validation MAE (q50 point reg):", mean_absolute_error(y_val, val_pred))



Quick validation MAE (q50 point reg): 36.23731797870682


In [7]:
predictions = pred_df[['SKU-index', 'Week_Index']].copy()

for q in QUANTILES:
    print(f"\nTraining quantile model alpha={q} ...")

    params = {
        "objective": "quantile",
        "metric": "quantile",
        "alpha": q,
        "learning_rate": 0.05,
        "num_leaves": 31,
        "seed": RANDOM_SEED,
        "verbose": -1
    }

    dtrain_q = lgb.Dataset(train_df[features], label=train_df[TARGET])

   
    model = lgb.train(
        params,
        dtrain_q,
        num_boost_round=500,
        callbacks=[lgb.log_evaluation(100)]  
    )

   
    model_fname = os.path.join(MODELS_DIR, f"lgb_q{int(q*100)}.pkl")
    joblib.dump(model, model_fname)



    pred_vals = model.predict(pred_df[features].fillna(0))
    colname = f"q{int(q*100)}_ml"
    predictions[colname] = pred_vals



Training quantile model alpha=0.1 ...

Training quantile model alpha=0.5 ...

Training quantile model alpha=0.9 ...


In [8]:
for c in predictions.columns:
    if c.startswith("q"):
        predictions[c] = predictions[c].clip(lower=0)



In [9]:
out_file = os.path.join("outputs", "forecast_ml.xlsx")
predictions.to_excel(out_file, index=False)

print(predictions.head(20))


     SKU-index  Week_Index   q10_ml     q50_ml      q90_ml
15           1          19  0.00000  40.447799  318.674306
33           2          19  0.00000   0.000000    0.000000
51           3          19  0.00000   2.297156    9.070812
69           4          19  0.00000   0.541287   21.207746
87           5          19  0.00000  25.694311  175.124117
105          6          19  0.70954  35.481413  163.101670
123          7          19  0.00000   7.076192   86.030563
141          8          19  0.00000   0.000000    0.000000
159          9          19  0.00000   0.000000    0.000000
177         10          19  0.00000  44.242505  166.002420
195         11          19  0.00000   0.862098    0.000000
213         12          19  0.00000   1.975973   12.168970
231         13          19  0.00000   2.396572    5.871597
249         14          19  0.00000   0.000000    0.540490
267         15          19  0.00000   0.862098    0.000000
285         16          19  0.00000   1.758717   10.3052

### part2 : train prophet model

In [10]:
import pandas as pd
from prophet import Prophet

df = pd.read_excel("weekly_aggregate.xlsx")

df = df.rename(columns={'week': 'ds', 'avg_demand': 'y'})


df['ds'] = pd.to_datetime('2025-01-21') + pd.to_timedelta((df['ds'] - 1) * 7, unit='D')




model = Prophet(weekly_seasonality=True, yearly_seasonality=False, daily_seasonality=False)
model.fit(df)


future = model.make_future_dataframe(periods=4, freq='W')
forecast = model.predict(future)


forecast_next = forecast.tail(1)[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
forecast_next.rename(columns={'yhat': 'predicted_avg_demand'}, inplace=True)




sku_template = pd.read_excel("sku_week_template.xlsx")


sku_template['predicted_avg_demand'] = forecast_next['predicted_avg_demand'].values[0]

sku_template.to_excel("outputs/forecast_prophet.xlsx", index=False)




21:45:08 - cmdstanpy - INFO - Chain [1] start processing
21:45:09 - cmdstanpy - INFO - Chain [1] done processing


### make D_used

In [11]:
import pandas as pd


ml = pd.read_excel("outputs/forecast_ml.xlsx")              
prophet = pd.read_excel("outputs/forecast_prophet.xlsx")   
mean3 = pd.read_excel("mean3.xlsx")                        


ml = ml.rename(columns={"SKU-index": "SKU"})


df = (
    ml.merge(prophet[["SKU", "predicted_avg_demand"]], on="SKU", how="left")
      .merge(mean3[["SKU", "mean"]], on="SKU", how="left")
)


important_skus = list(range(1, 10)) + list(range(35, 39))


def compute_final_forecast(row):
    if row["SKU"] in important_skus:
   
        ml_val = row["q90_ml"]
        prophet_val = row["predicted_avg_demand"]
    else:
     
        ml_val = row["q50_ml"]
        prophet_val = row["predicted_avg_demand"]
    mean_val = row["mean"]
    return 0.1 * ml_val + 0 * prophet_val + 0.9 * mean_val

df["D_used"] = df.apply(compute_final_forecast, axis=1)


df_out = df[["SKU", "Week_Index", "D_used"]]
df_out.to_excel("outputs/D_used_week19.xlsx", index=False)




### make D_input

In [12]:
import pandas as pd


d_used = pd.read_excel("outputs/D_used_week19.xlsx")
mean_long = pd.read_excel("mean per week.xlsx")


d_used = d_used.rename(columns={"SKU": "SKU-index"})
mean_long = mean_long.rename(columns={"SKU": "SKU-index"})


df = d_used.merge(mean_long, on="SKU-index", how="left")


df["D_input"] = 0.8 * df["mean"] + 0.2 * df["D_used"]


df_out = df[["SKU-index", "Week_Index", "D_input"]]
df_out.to_excel("outputs/D_input_week19.xlsx", index=False)


